In [ ]:

import hail as hl

from hail.plot import output_notebook, show

In [ ]:
pip install hail

     |████████████████████████████████| 93.2 MB 13 kB/s 
     |████████████████████████████████| 73 kB 1.8 MB/s 
     |████████████████████████████████| 107 kB 49.9 MB/s 
     |████████████████████████████████| 51 kB 506 kB/s 
     |████████████████████████████████| 59 kB 6.4 MB/s 
     |████████████████████████████████| 131 kB 35.3 MB/s 
     |████████████████████████████████| 7.7 MB 26.9 MB/s 
     |████████████████████████████████| 45 kB 2.6 MB/s 
     |████████████████████████████████| 46 kB 3.5 MB/s 
     |████████████████████████████████| 1.3 MB 47.2 MB/s 
     |████████████████████████████████| 9.5 MB 55.1 MB/s 
     |████████████████████████████████| 32.4 MB 75 kB/s 
     |████████████████████████████████| 212.4 MB 68 kB/s 
     |████████████████████████████████| 61 kB 6.9 MB/s 
     |████████████████████████████████| 294 kB 41.8 MB/s 
     |████████████████████████████████| 142 kB 36.8 MB/s 
     |████████████████████████████████| 179 kB 48.1 MB/s 
     |██████████████████████

In [ ]:
hl.init()
output_notebook()


Running on Apache Spark version 3.1.2
SparkUI available at http://20cf418ff9ee:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.72-cfce5e858cab
LOGGING: writing to /content/hail-20210720-1837-0.2.72-cfce5e858cab.log


In [ ]:
! ls -1 resources/


ls: cannot access 'resources/': No such file or directory


In [ ]:
hl.utils.get_1kg('data/')


2021-07-20 18:37:30 Hail: INFO: downloading 1KG VCF ...
  Source: https://storage.googleapis.com/hail-tutorial/1kg.vcf.bgz
2021-07-20 18:37:30 Hail: INFO: importing VCF and writing to matrix table...
2021-07-20 18:37:36 Hail: INFO: Coerced sorted dataset
2021-07-20 18:37:46 Hail: INFO: wrote matrix table with 10879 rows and 284 columns in 16 partitions to data/1kg.mt
    Total size: 15.97 MiB
    * Rows/entries: 15.97 MiB
    * Columns: 1.31 KiB
    * Globals: 11.00 B
    * Smallest partition: 708 rows (1007.66 KiB)
    * Largest partition:  680 rows (1.01 MiB)
2021-07-20 18:37:46 Hail: INFO: downloading 1KG annotations ...
  Source: https://storage.googleapis.com/hail-tutorial/1kg_annotations.txt
2021-07-20 18:37:46 Hail: INFO: downloading Ensembl gene annotations ...
  Source: https://storage.googleapis.com/hail-tutorial/ensembl_gene_annotations.txt
2021-07-20 18:37:46 Hail: INFO: Done!


In [ ]:
hl.import_vcf('data/1kg.vcf.bgz').write('data/1kg.mt', overwrite=True)


2021-07-20 18:37:58 Hail: INFO: Coerced sorted dataset
2021-07-20 18:38:05 Hail: INFO: wrote matrix table with 10879 rows and 284 columns in 1 partition to data/1kg.mt
    Total size: 16.09 MiB
    * Rows/entries: 16.09 MiB
    * Columns: 1.31 KiB
    * Globals: 11.00 B
    * Smallest partition: 10879 rows (16.09 MiB)
    * Largest partition:  10879 rows (16.09 MiB)


In [ ]:
mt = hl.read_matrix_table('data/1kg.mt')


In [ ]:
table = hl.import_table('data/1kg_annotations.txt', impute=True).key_by('Sample')


2021-07-20 18:38:13 Hail: INFO: Reading table to impute column types
2021-07-20 18:38:15 Hail: INFO: Finished type imputation
  Loading field 'Sample' as type str (imputed)
  Loading field 'Population' as type str (imputed)
  Loading field 'SuperPopulation' as type str (imputed)
  Loading field 'isFemale' as type bool (imputed)
  Loading field 'PurpleHair' as type bool (imputed)
  Loading field 'CaffeineConsumption' as type int32 (imputed)


In [ ]:
mt = hl.read_matrix_table('data/1kg.mt')
mt = mt.annotate_cols(pheno = table[mt.s])
mt = hl.sample_qc(mt)
mt = mt.filter_cols((mt.sample_qc.dp_stats.mean >= 4) & (mt.sample_qc.call_rate >= 0.97))
ab = mt.AD[1] / hl.sum(mt.AD)

In [ ]:
filter_condition_ab = ((mt.GT.is_hom_ref() & (ab <= 0.1)) |
                        (mt.GT.is_het() & (ab >= 0.25) & (ab <= 0.75)) |
                        (mt.GT.is_hom_var() & (ab >= 0.9)))
mt = mt.filter_entries(filter_condition_ab)
mt = hl.variant_qc(mt)
mt = mt.filter_rows(mt.variant_qc.AF[1] > 0.01)

eigenvalues, pcs, _ = hl.hwe_normalized_pca(mt.GT)

mt = mt.annotate_cols(scores = pcs[mt.s].scores)

2021-07-20 18:38:40 Hail: INFO: hwe_normalized_pca: running PCA using 9087 variants.
2021-07-20 18:38:46 Hail: INFO: pca: running PCA with 10 components...


In [ ]:
mt.show()

,
locus,alleles
locus<GRCh37>,array<str>
1:904165,"[""G"",""A""]"
1:1563691,"[""T"",""G""]"
1:1707740,"[""T"",""G""]"
1:2284195,"[""T"",""C""]"
1:2779043,"[""T"",""C""]"
1:2944527,"[""G"",""A""]"
1:3761547,"[""C"",""A""]"
1:3803755,"[""T"",""C""]"


In [ ]:
gwas = hl.linear_regression_rows(
    y=mt.pheno.CaffeineConsumption,
    x=mt.GT.n_alt_alleles(),
    covariates=[1.0, mt.pheno.isFemale, mt.scores[0], mt.scores[1], mt.scores[2]])

2021-07-20 18:39:57 Hail: INFO: linear_regression_rows: running on 250 samples for 1 response variable y,
    with input variable x, and 5 additional covariates...


In [ ]:
mt.GT.n_alt_alleles().show()

,,,,
,,'HG00096','HG00099','HG00105'
locus,alleles,,,
locus<GRCh37>,array<str>,int32,int32,int32
1:904165,"[""G"",""A""]",0,0,0
1:1563691,"[""T"",""G""]",NA,0,0
1:1707740,"[""T"",""G""]",1,1,1
1:2284195,"[""T"",""C""]",2,1,1
1:2779043,"[""T"",""C""]",1,1,2
1:2944527,"[""G"",""A""]",0,1,NA
1:3761547,"[""C"",""A""]",0,0,0


In [ ]:
import pandas as pd
 t = hl.Table.from_pandas(df) 


In [ ]:
mt.show()

,
locus,alleles
locus<GRCh37>,array<str>
1:904165,"[""G"",""A""]"
1:1563691,"[""T"",""G""]"
1:1707740,"[""T"",""G""]"
1:2284195,"[""T"",""C""]"
1:2779043,"[""T"",""C""]"
1:2944527,"[""G"",""A""]"
1:3761547,"[""C"",""A""]"
1:3803755,"[""T"",""C""]"


In [ ]:
input=hl.Table.to_pandas(mt)

AttributeError: ignored

In [ ]:
mt.GT.n_alt_alleles().show()

,,,,
,,'HG00096','HG00099','HG00105'
locus,alleles,,,
locus<GRCh37>,array<str>,int32,int32,int32
1:904165,"[""G"",""A""]",0,0,0
1:1563691,"[""T"",""G""]",NA,0,0
1:1707740,"[""T"",""G""]",1,1,1
1:2284195,"[""T"",""C""]",2,1,1
1:2779043,"[""T"",""C""]",1,1,2
1:2944527,"[""G"",""A""]",0,1,NA
1:3761547,"[""C"",""A""]",0,0,0


In [ ]:
p = hl.plot.qq(gwas.p_value)
show(p)

2021-07-19 17:08:45 Hail: INFO: Ordering unsorted dataset with network shuffle


In [ ]:
p = hl.plot.manhattan(gwas.p_value)
show(p)